# ZephIR Demo

[https://github.com/venkatachalamlab/ZephIR](https://github.com/venkatachalamlab/ZephIR)

This notebook is a demo to illustrate a typical ZephIR workflow and is accompanied by a general [guide](https://github.com/venkatachalamlab/ZephIR/blob/main/docs/Guide-ZephIR.md) and [tutorial video](https://youtu.be/4O9aIftvoqM).

This notebook walks through the steps to analyze the provided short movie of semi-immobilized *C. elegans* with fluorescent IFT-20 expression. We will track the fluorescent neuron nuclei and extract their activity traces.

In [ ]:
# Import ZephIR package
import zephir
from zephir.main import *

# ZephIR is mostly run with terminal commands
# Subprocess library will be used to emulate this
import subprocess

In [ ]:
# Define path to dataset directory
dataset = Path('__file__').parent.absolute() / 'data'

## Setting up and preprocessing
To get the most of ZephIR, it is recommended that the data is preprocessed such that noise, photobleaching, and global translation & rotation are reduced. The demo dataset has reduced motion (global translation has been reduced by 50%), but no further preprocessing has been performed.


## getters.py
Users attempting to work with data with different structures and IO bindings can edit a `getters.py` file that consolidate key IO functions. The demo dataset is fully compatible with the default getters and does not require any further editing.

## Annotating reference frame

ZephIR uses fully-annotated (all keypoints to be tracked are annotated) reference frames to seed the registration and tracking process. The built-in recommendation function uses k-medoids clustering to find N median frames that are most similar to most of the frames in the data. After determining the frames you want to use as reference frames, the included annotator GUI allows users to create annotations for those frames.

Run the following cells to create new annotations. Alternatively, to skip to the next step, copy the file `annotations-2.h5` in the backup folder into this folder and rename it to `annotations.h5`. This file contains annotations for the first recommended frame (plus one frame that includes a neuron that is not visible in the recommended frame).

In [ ]:
# Determine the best frames to annotate as reference frames
subprocess.run(f'recommend_frames --dataset={dataset}', shell=True)

In [ ]:
# Launch annotator to create annotations for reference frames
# Run this cell then open in a web browser: localhost:5001
subprocess.run(f'annotator --dataset={dataset} --port=5001', shell=True)

## Running ZephIR

Run the following cell to run ZephIR with either the provided or the newly created annotations. ZephIR will use parameter and options outlined in `args.json`. Feel free to play with these parameters to test how they affect tracking quality.

ZephIR will create a movie annotated with the tracking results (`annotated.mp4`). This can be used to check results at a glance, but the annotator can be used for more detailed and careful verification of results. Any annotation with the provenance `ZEIR` is a tracking result automatically generated by ZephIR.

In [ ]:
# Launch ZephIR to track keypoints as defined in fully-annotated reference frames
# User-tunable options are defined in the included args.json file

# Non-default arguments include:
# channel=1
# fovea_sigma=5.0
# gamma=1
# grid_shape=35
# lambda_n=0.1
# lambda_n_mode=norm
# lr_ceiling=0.06
# lr_floor=0.01
# motion_predict=True
# z_compensator=4.0

subprocess.run(f'zephir --dataset={dataset} --load_args=True', shell=True)

## Verifying results and correcting errors

Launch the annotator for a detailed and careful verification of ZephIR tracking results.

Fixing badly tracked frames by:

1. Verify a few correct tracking results by changing their provenance from `ZEIR` to another (`ANTT` by default). The macro `change_provenance` can do this efficiently.
2. Delete a few incorrect tracking results and insert new annotations for those keypoint at the correct locations.
3. Run the macro `update_frame` from the annotator GUI to run ZephIR for any remaining keypoints in the frame. It will use both verified correct results and fixed incorrect ones to improve tracking quality for the rest of the keypoints.

Alternatively, to skip to the next step, copy the file `annotations-g.h5` in the backup folder into this folder and rename it to `annotations.h5`. This file contains annotations for the first recommended frame, one frame including a neuron that is not visible in the recommended frame, partial annotations for a badly tracked frame (t=23), and ZephIR results for the rest.

In [ ]:
# Launch annotator to verify results and correct errors
# Run this cell then open in a web browser: localhost:5001
subprocess.run(f'annotator --dataset={dataset} --port=5001', shell=True)

## Extracting neuron activity traces

Run the following cell to extract traces of neuronal activity. Neuronal activity, expressed as fluorescent intensity in the confocal microscopy recording, is calculated as the average of 16 maximal pixel intensities in a 3x9x9 crop around the tracked coordinates in each frame (after masking out any neighboring neurons that may end up in the crop). Traces extracted using tracking results in `annotations-g.h5` in the backup folder is also available in the backup folder as `traces.png`.

In [ ]:
# Launch trace extraction to extract fluorescence intensity using tracked centers
subprocess.run(f'extract_traces --dataset={dataset} --channel=1 --debleach=False', shell=True)